# PHYS3116

## Stellar Model

A stellar model is a set of computer programs that calculates the interior structure of a star. It does this by integrating the stellar structure equations to calculate the radius, luminosity, pressure, and temperature across a range of mass points from the centre of a star to its surface. For a star with a given mass and composition there is one unique solution for these equations.

We will be constructing a model for a main-sequence star. To build this stellar model we will need a number of ingredients and subroutines. 

Here we will often be using a mass of $M_{tot} = 1.2 M_\odot$ (i.e. 20% more massive than the Sun), and a composition of:
<br><br>$X =$ mass fraction of hydrogen $= 0.7$,
<br>$Y =$ mass fraction of helium $= 0.2$,
<br>$Z =$ mass fraction of all other elements combined $= 0.1$.

Note that in the second assignment you may be asked to vary these to model stars with different masses and compositions.

Our stellar model will be written in python (using jupyter notebook). The code provided here is deliberately not concise, not optimised, and could be improved in many ways. It is provided as a working version such that if you are new to python you will have a stellar model that I will show you how to use. If you are already comfortable with python, you may use this code or develop your own version that you like better.

The first task is to calculate the pressure, density, opacity, and energy generation rate of our star.

In [ ]:
# Import the libraries we need.

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

%matplotlib inline

In [ ]:
# Define physical constants.
# All values come from Appendix A of Carroll & Ostlie

a = 7.565767E-16               # Radiation constant, J mol^-3 K^-4
G = 6.67428E-11                # Gravitational constant, m^3 kg^-1 s^-2
c = 2.99792458E8               # Speed of light, m s^-1
boltzmann = 1.3806504E-23      # Boltzmann's constant, J K^-1
H_mass = 1.673532499E-27       # Mass of a hydrogen atom, kg
sigma = 5.670400E-8            # Stefan-Boltzmann constant, W m^-2 K^-4
R = boltzmann/H_mass           # Ideal gas constant, J K^-1 kg^-1

# Solar parameters:

Msun = 1.989E30                # Mass of Sun
Rsun = 6.95508E8               # Radius of Sun
Lsun = 3.839E26                # Luminosity of Sun

# Stellar parameters:

Mtot = 1.2*Msun                # 20% more massive than the Sun 
mfraction = 1e-8               # Fractional mass of core 
X = 0.7                        # Mass fraction of hydrogen
Y = 0.2                        # Mass fraction of helium
log_pc_i = 17.1                # Initial estimate for pressure at the centre of the star (slightly higher than Sun)
log_tc_i = 7.4                 # Initial estimate for temperature at the centre of the star (slightly higher than Sun)
log_rs_i = 8.85                # Initial estimate for full radius of the star (slightly higher than Sun)
log_ls_i = 26.6                # Initial estimate for full luminosity of the star (slightly higher than Sun)

### Pressure and Density

The total pressure $P$ is the sum of the gas pressure and the radiation pressure. It is related to the temperature $T$, density $\rho$, and mean molecular weight $\mu$ by the equation:

$P = \frac{R}{\mu}\rho T + \frac{a}{3} T^4$

We can rearrange this equation to give the density $\rho$, as it relates to the temperature, total pressure, and stellar composition:

$\rho = \frac{\mu}{RT}(P-\frac{aT^4}{3})$

The ratio of gas pressure to total pressure $\beta$ is important for determining the temperature gradient in the star. Note that gas pressure is most important in low mass stars, while for more massive stars radiation pressure will start to dominate.

We will write some functions to accept input values for $X$, $Y$, $\log(T)$, and $\log(P)$, and return values for $\rho$ and $\beta$.

### Energy Generation

The energy generation rate $\epsilon$ is the sum of the energy generated by the proton-proton chain and the CNO cycle, both of which convert hydrogen into helium. Our model is slightly simplified to only include the most common reaction pathways in the proton-proton chain and the CNO cycle and to ignore some details of the shape of the Gamow peak, so we can write:

$\epsilon_{pp} = 0.241\times\rho X^2T_6^{-2/3}e^{-33.8T_6^{-1/3}}$

and

$\epsilon_{CNO} = 8.67\times10^{20}\times\rho X X_{CNO}T_6^{-2/3}e^{-152.28T_6^{-1/3}}$

In [ ]:
# Define functions from Part 1:

def mean_molecular_weight(X,Y):
    """
    Calculates the mean molecular weight.
    
    Parameters
    ----------
    X: float
        Mass fraction of hydrogen
    Y: float
        Mass fraction of helium
        
    Returns
    -------
    mu: float
        Mean molecular weight
    """
    
    Z = (1 - X - Y)
    mu = 1/((2*X)+(0.75*Y)+(0.5*Z))
    
    return mu

def density(P,T,mu):
    """
    Calculates the density.
    
    Parameters
    ----------
    P: float
        Total pressure
    T: float
        Temperature
    mu: float
        Mean molecular weight
        
    Returns
    -------
    rho: float
        Density
    """
    
    rho = (mu/(R*T))*(P-(a*T**4)/3)
    
    return rho

def pressure_to_density_beta(X, Y, logT, logP):
    """
    Accepts values for X, Y, log(T), and log(P), and returns values for rho and beta.
    """
    
    mu = mean_molecular_weight(X,Y)
    rho = density(10**logP, 10**logT, mu)
    gas_pressure = (R/mu)*rho*(10**logT)
    beta = gas_pressure/(10**logP)
     
    return rho, beta

def opacity(logT, logrho, X, Y):
    T = 10**logT
    rho = 10**(logrho - 3)        # -3 converts input quantities from SI to cgs units to support coefficients below
    
    #Kramer's opacity law: kappa is proportional to rho and T^-3.5
    k_bf = (4.34e25*(1-X-Y)*(1+X)*rho*np.power(T,-3.5))/(2.82*np.power(rho*(1+X),0.2))  #bound-free absorption
    k_ff = 3.68e22*(X+Y)*(1+X)*rho*np.power(T,-3.5)                                     #free-free absorption
    k_es = 0.2*(1+X)                                                                    #electron scattering
    
    return np.log10(k_bf + k_ff + k_es) - 1                                             # -1 to convert to SI units

def energy_generation(logrho,logT,X,Y):
    t6 = (10**logT)/(1E6)
    rho = 10**logrho
    Xcno = 0.7263*(1-X-Y)
    
    eps_pp = 0.241*rho*np.power(X,2)*np.power(t6,-2/3)*np.exp(-33.8/np.power(t6,1/3))
    eps_CNO = 8.67E20*rho*X*Xcno*np.power(t6,-2/3)*np.exp(-152.28/np.power(t6,1/3))

    eps = eps_pp + eps_CNO
    
    return eps

### Centre of star:

We are going to use the four differential stellar structure equations. We can't begin our model at the very centre of the star, because all of the derivatives are zero there. Instead we will begin a small distance from the centre, and use a Taylor series expansion to solve the equations. If we choose our small distance to be $m = 0.01\%$ of the mass of the star (with a stellar mass of $M_{star} = 1.2 \times M_{\odot}$), the following equations give the values for the radius and luminosity:

$R = \left(\frac{3}{4\pi\rho}\right)^{1/3}m^{1/3}$

$L = \epsilon m$

We need to make an initial estimate of the central pressure and temperature. We will refine these estimates towards their true values as the stellar model program runs. For now, because our star is slightly more massive than the Sun, we choose values that are slightly higher than those for the Sun: $\log(P_c) = 17.1$, $\log(T_c) = 7.4$. Now to calculate the pressure and temperature at a small distance from the centre of the star, we add small offsets. Our pressure is now expressed as:

$P = P_c - \frac{3G}{8\pi}\left(\frac{4\pi\rho}{3}\right)^{4/3}m^{2/3}$

There are two possible offsets for the temperature, depending on which is steeper of the adiabatic temperature gradient, and the radiative temperature gradient.

The adiabatic temperature gradient is $\nabla_{ad} = \frac{\frac{(1-\beta)(4+\beta)}{\beta^2}+1}{\frac{4(1-\beta)(4+\beta)}{\beta^2}+2.5}$

The radiative temperature gradient is $\nabla_{rad} = \frac{3\kappa LP}{16\pi GacT_c^4m}$

If $\nabla_{ad}$ is the smaller value, the the star is convective and the temperature at a small distance from the centre of the star is calculated as:

$\ln(T) = \ln(T_c) - \left(\frac{\pi}{6}\right)^{1/3}G\frac{\nabla_{ad}\rho^{4/3}}{P_c}m^{2/3}$

On the other hand, if $\nabla_{rad}$ is smaller, then the star is radiative and the temperature at a small distance from the star is calculated as:

$T^4 = T_c^4 - \frac{1}{2ac}\left(\frac{3}{4\pi}\right)^{2/3}\kappa\epsilon\rho^{4/3}m^{2/3}$

### Surface of star:

Here we need to make an initial estimate of the full radius and luminosity of the star. Like the central temperature and pressure, these values will be refined towards their true values as the stellar model program runs. Once again, because our star is slightly more massive than the Sun, we will choose values that are slightly higher than those for the Sun: $\log(R_s) = 8.85$, $\log(L_s) = 26.6$. Now we can calculate the temperature at the surface of the star $T_s$ by rearranging the expression for the star's total luminosity:

$L_s = 4\pi R_s^2\sigma T_s^4$

Where $\sigma$ is the Stefan-Boltzmann constant. This gives:

$T_s = \left(\frac{L_s}{4\pi R_s^2\sigma}\right)^{1/4}$

The pressure at the surface is a little more difficult to find. We have:

$P_s = \frac{GM_{tot}}{R_s^2}\frac{2}{3}\frac{1}{\kappa}$

Where $M_{tot}$ is the total mass of the star ($1.2\times M_\odot$). However to interpolate the opacity $\kappa$ we need to know density $\rho$, and to calculate $\rho$ we need to know the pressure at the surface $P_s$. To get around this, we will start with a trial value of $P_s = 10^3$, calculate $\rho$, find $\kappa$, and then calculate $P_s$. If our calculated value for $P_s$ is within $5\%$ of the trial value we will accept this as the right answer. If it is not, we will need to adjust the trial value toward the calculated value and try again.

The function below accepts input values of $M_{tot}$, $\log(R_s)$, and $\log(L_s)$, and uses these values to calculate and return values for $T_s$ and $P_s$. Our function will include a loop that takes a trial value of $P_s$, calculates $\rho$, finds $\kappa$, and calculates $P_s$, and then iterates until the trial and calculated values of $P_s$ are within $5\%$ of each other.

In [ ]:
def centre(m, log_pc_i, log_tc_i, X, Y):
    m = mfraction*Mtot
    p_c_i = 10**log_pc_i
    t_c_i = 10**log_tc_i
    
    ## Incorporate previously defined functions ##
    rho_c, beta_c = pressure_to_density_beta(X=X, Y=Y, logT=log_tc_i, logP=log_pc_i)
    epsilon_c = energy_generation(logrho=np.log10(rho_c), logT=log_tc_i, X=X, Y=Y)
    log_kappa_c = opacity(logT=log_tc_i, logrho=np.log10(rho_c), X=X, Y=Y)
    kappa_c = 10**log_kappa_c
    
    ## Calculate radius, luminosity, and pressure ##
    r_c = np.power(0.75/(np.pi*rho_c), 1/3)*np.power(m, 1/3)
    l_c = epsilon_c*m
    p_c = p_c_i - (3*G/(8*np.pi))*np.power((4*np.pi*rho_c)/3, 4/3)*np.power(m, 2/3)
    
    ## Define temperature gradients ##
    grad_ad = (((1-beta_c)*(4+beta_c))/(beta_c**2) + 1)/(((4*(1-beta_c)*(4+beta_c))/(beta_c**2))+2.5)
    grad_rad = (3*kappa_c*l_c*p_c)/(16*np.pi*G*a*c*m*np.power(t_c_i, 4))
    
    ## Calculate temperature for convective/radiative core ##
    if grad_rad < grad_ad:
        t_4 = np.power(t_c_i, 4) - (0.5/(a*c))*np.power(0.75/np.pi, 2/3)*kappa_c*epsilon_c*np.power(rho_c, 4/3)*np.power(m, 2/3)
        t_c = np.power(t_4, 1/4)
        #print('Core is radiative!')
    else:
        ln_t = np.log(t_c_i) - np.power(np.pi/6, 1/3)*G*grad_ad*np.power(rho_c, 4/3)*np.power(m, 2/3)/p_c_i
        t_c = np.exp(ln_t)
        #print('Core is convective!')
    
    return r_c, l_c, p_c, t_c

def surface(m, log_rs_i, log_ls_i, init_ps):
    m = Mtot
    r_s = 10**log_rs_i
    l_s = 10**log_ls_i
    t_s = np.power(l_s/(4*np.pi*np.power(r_s, 2)*sigma), 1/4)
    
    trial_ps = init_ps
    candidate_ps = 0
    diff = 0

    while np.abs((trial_ps - candidate_ps)/trial_ps) > 0.05:
        diff = trial_ps - candidate_ps
        trial_ps = trial_ps - 0.1*diff
        rho_s, beta_s = pressure_to_density_beta(X=X, Y=Y, logT=np.log10(t_s), logP=np.log10(trial_ps))
        log_kappa_s = opacity(logT=np.log10(t_s), logrho=np.log10(rho_s), X=X, Y=Y)
        kappa_s = 10**log_kappa_s
        candidate_ps = (2*G*m)/(3*kappa_s*(r_s**2))
        
    return r_s, l_s, candidate_ps, t_s
    
def partial_derivatives(rlpt, m):
    # Unpack rlpt
    r = rlpt[0]
    l = rlpt[1]
    p = rlpt[2]
    t = rlpt[3]
    
    # Convert to log10 as this is used in our previous functions
    logl = np.log10(l)
    logp = np.log10(p)
    logt = np.log10(t)
    
    # Use previous functions to calculate rho, beta, epsilon, and kappa
    rho, beta = pressure_to_density_beta(X=X, Y=Y, logT=logt, logP=logp)
    logrho = np.log10(rho)
    epsilon = energy_generation(logrho=logrho, logT=logt, X=X, Y=Y)
    log_kappa = opacity(logT=logt, logrho=logrho, X=X, Y=Y)
    kappa = 10**log_kappa
    
    # Calculate grad as the lesser of grad_rad and grad_ad
    grad_ad = (((1-beta)*(4+beta))/(beta**2) + 1)/(((4*(1-beta)*(4+beta))/(beta**2))+2.5)
    grad_rad = (3*kappa*l*p)/(16*np.pi*G*a*c*m*np.power(t, 4))
    grad = min(grad_rad, grad_ad)
    
    # Find the derivatives in our equations of stellar structure
    drdm = 1/(4*np.pi*np.power(r, 2)*rho)
    dldm = epsilon
    dpdm = -(G*m)/(4*np.pi*np.power(r, 4))
    dtdm = -(G*m*t*grad)/(4*np.pi*p*np.power(r, 4))
    
    return [drdm, dldm, dpdm, dtdm]

We will now integrate the stellar structure equations outward from the centre and inward from the surface. The goal is for the two integrations to meet at $m = \frac{2}{3}M_{tot}$, though this probably won't be the case using our initial estimates. Here we will calculate the offsets between the integrations at the meeting point. These can then be used in Part 4 to improve our model.

To do these integrations, we will use scipy.integrate.odeint. It can integrate all four stellar structure equations at once (what we will do here), or you can choose to integrate them separately. As input, we will need our initial conditions ($R_s$, $L_s$, $P_c$, and $T_c$), our function from Part 2 that returns the gradients we want to integrate ($\frac{\partial R}{\partial m}$, $\frac{\partial L}{\partial m}$, $\frac{\partial P}{\partial m}$, and $\frac{\partial T}{\partial m}$), and a set of mass points at which we want values for $R$, $L$, $P$, and $T$ to be calculated (in this case from $m$ to $\frac{2}{3}M_{tot}$, or from $M_{tot}$ to $\frac{2}{3}M_{tot}$). We will have the integrations run one extra step past the meeting point (for reasons that will become clear in Part 4).

For the outward integration we will set the mass step size to be $(\frac{2}{3}M_{tot} - m)/10^4$. For the inward integration we need to add a bit more complexity. The gradients in $T$ and $R$ are very steep at the outside edge of the star (i.e. at the surface), so we need to use very small integration steps at first. We can then make these steps larger (to cut down on computation) once we get a small distance into the star. Here we will divide the outermost $0.01\%$ of the mass into $10^4$ steps, and the remaining inward integration into $10^4$ steps. Note that this may require some adjustment for different models.

Let's write a program that takes the initial estimates as inputs, calculates initial centre and surface values and derivatives for $R$, $L$, $P$, and $T$ using our earlier functions, chooses step sizes, and runs our two integrations. The program will also have the option to plot figures showing the values of $R$, $L$, $P$, and $T$ at all mass points, and will return an array of the offsets between the integrations at the meeting point.

In [ ]:
def model(rlpt, do_plot):
    
    log_rs, log_ls, log_pc, log_tc = np.log10(rlpt)
    
    meeting_point = 0.67
    
    # Choose mass steps for the integration from the inner point,
    # integrating outwards to a mass of (Mtot*meeting_point)
    
    mgrid_c = Mtot*np.linspace(mfraction, meeting_point, 10000)
    mgrid_c = np.append(mgrid_c, mgrid_c[-1] + np.diff(mgrid_c)[0])
    mass_step_out = mgrid_c
    
    # Set up the initial values at the centre
    
    rlpt_c = centre(m=mfraction,log_pc_i=log_pc,log_tc_i=log_tc,X=X,Y=Y)
    
    # Integrate from the centre outward
    
    solve_outwards = odeint(partial_derivatives, rlpt_c, mgrid_c)
    
    # Choose mass steps for the integration from the surface,
    # integrating inwards to a mass of (Mtot*meeting_point).
    # We take tiny steps when we are close to the surface,
    # since change is rapid there.
    
    mgrid_s1 = np.linspace(Mtot, Mtot*(2.9999/3), 10000)
    mgrid_s2 = np.linspace(Mtot*(2.9999/3), meeting_point*Mtot, 10000)
    mgrid_s2 = np.append(mgrid_s2, mgrid_s2[-1] + np.diff(mgrid_s2)[0])
    mgrid_s = np.concatenate([mgrid_s1[:-1], mgrid_s2], axis=0)
    mass_step_in = mgrid_s
    
    # Set up the initial values at the surface
    
    rlpt_s = surface(m=Mtot, log_rs_i=log_rs, log_ls_i=log_ls, init_ps=1e3)
    
    # Integrate from the surface inwards
    
    solve_inwards = odeint(partial_derivatives, rlpt_s, mgrid_s)
    
    # Calculate the offsets at the meeting point
    
    delta_r = solve_outwards[-2][0] - solve_inwards[-2][0]    # [-2] is meeting point, [0] is radius
    delta_l = solve_outwards[-2][1] - solve_inwards[-2][1]    # '' [1] is luminosity
    delta_p = solve_outwards[-2][2] - solve_inwards[-2][2]    # '' [2] is pressure
    delta_t = solve_outwards[-2][3] - solve_inwards[-2][3]    # '' [3] is temperature
    
    step_r = max([np.abs(solve_outwards[-1][0] - solve_outwards[-2][0]),np.abs(solve_inwards[-1][0]-solve_inwards[-2][0])])
    step_l = max([np.abs(solve_outwards[-1][1] - solve_outwards[-2][1]),np.abs(solve_inwards[-1][1]-solve_inwards[-2][1])])
    step_p = max([np.abs(solve_outwards[-1][2] - solve_outwards[-2][2]),np.abs(solve_inwards[-1][2]-solve_inwards[-2][2])])
    step_t = max([np.abs(solve_outwards[-1][3] - solve_outwards[-2][3]),np.abs(solve_inwards[-1][3]-solve_inwards[-2][3])])
    
    # Calculate values for density and energy generation so we can plot these as well
    
    pres_c = np.log10(solve_outwards[:,2])
    temp_c = np.log10(solve_outwards[:,3])
    pres_s = np.log10(solve_inwards[:,2])
    temp_s = np.log10(solve_inwards[:,3])

    rho_cen, beta_cen = pressure_to_density_beta(X=X, Y=Y, logT=temp_c, logP=pres_c)
    rho_sur, beta_sur = pressure_to_density_beta(X=X, Y=Y, logT=temp_s, logP=pres_s)
    e_cen = energy_generation(logrho=np.log10(rho_cen), logT=temp_c, X=X, Y=Y)
    e_sur = energy_generation(logrho=np.log10(rho_sur), logT=temp_s, X=X, Y=Y)
    
    
    # If requested, make some plots
    
    if do_plot:
        
        # Plots showing step distribution for each integration
        
        fig = plt.figure(figsize=(15,7.5))
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)

        ax1.scatter(np.arange(len(mgrid_c)), mgrid_c)
        ax1.axvline(x=10000, ls='--', c='k')

        ax2.scatter(np.arange(len(mgrid_s)), mgrid_s)
        ax2.axvline(x=0, ls='--')
        ax2.axvline(x=10000, ls='--')
        ax2.axvline(x=20000, ls='--', c='k')
        ax2.text(x= 1500, y = 1.9e30, s='Outermost Surface')
        
        ax1.set_xlabel('Outward Steps')
        ax1.set_ylabel('Mass of Star')
        ax2.set_xlabel('Inward Steps')
        ax2.set_ylabel('Mass of Star')
        plt.show()
        
        # Plots showing R, L, P, T against M
        
        fig = plt.figure(figsize=(15,15))
        ax1 = fig.add_subplot(321)
        ax2 = fig.add_subplot(322)
        ax3 = fig.add_subplot(323)
        ax4 = fig.add_subplot(324)
        ax5 = fig.add_subplot(325)
        ax6 = fig.add_subplot(326)
        
        ax1.plot(mass_step_out/Msun, solve_outwards[:,0]/Rsun)
        ax1.plot(mass_step_in/Msun, solve_inwards[:,0]/Rsun)
        ax1.set_xlabel('Mass [solar masses]')
        ax1.set_ylabel('Radius [solar radii]')
        
        ax2.plot(mass_step_out/Msun, solve_outwards[:,1]/Lsun)
        ax2.plot(mass_step_in/Msun, solve_inwards[:,1]/Lsun)
        ax2.set_xlabel('Mass [solar masses]')
        ax2.set_ylabel('Luminosity [solar luminosities]')
        
        ax3.plot(mass_step_out/Msun, solve_outwards[:,2])
        ax3.plot(mass_step_in/Msun, solve_inwards[:,2])
        ax3.set_xlabel('Mass [solar masses]')
        ax3.set_ylabel('Pressure')
        
        ax4.plot(mass_step_out/Msun, solve_outwards[:,3])
        ax4.plot(mass_step_in/Msun, solve_inwards[:,3])
        ax4.set_xlabel('Mass [solar masses]')
        ax4.set_ylabel('Temperature')
        
        ax5.plot(mass_step_out/Msun, rho_cen)
        ax5.plot(mass_step_in/Msun, rho_sur)
        ax5.set_xlabel('Mass [solar masses]')
        ax5.set_ylabel('Density')
        
        ax6.plot(mass_step_out/Msun, e_cen)
        ax6.plot(mass_step_in/Msun, e_sur)
        ax6.set_xlabel('Mass [solar masses]')
        ax6.set_ylabel('Energy generation')
        
        plt.show()
    
    return (np.array([delta_r, delta_l, delta_p, delta_t]), np.array([step_r, step_l, step_p, step_t]))  

## Part 4:

The final step in making our stellar model will call upon the functions we have already written to set up the initial values and carry out the integrations from the centre and surface of the star to the meeting point at $\frac{2}{3}M_{tot}$. We will add a new section that uses the results of these integrations and the calculated offsets between them to adjust the initial conditions. The program will iterate through this process until the integrations meet to within an appropriate tolerance.

Adjusting the initial conditions is done by calculating partial derivatives to measure the sensitivity of the mismatches to the initial conditions. We do this by changing each of the four initial estimates ($R_s$, $L_s$, $P_c$, and $T_c$) by a small amount ($1\%$ is a good change), one at a time. We then run the integrations to the meeting point and see how much the mismatches change. For example, by re-running the integration with ($1.01 \times R_s$, $L_s$, $P_c$, and $T_c$) as the inputs, we can calulate $\frac{\partial\Delta R}{\partial R_s}$ as $\frac{\Delta R(\text{new integration}) - \Delta R(\text{original integration})}{0.01\times R_s}$. Our other partial derivatives $\frac{\partial\Delta L}{\partial R_s}$, $\frac{\partial\Delta P}{\partial R_s}$, and $\frac{\partial\Delta T}{\partial R_s}$ are calculated similarly.

We will write code to run four new integrations, varying each of the inputs in turn by $1\%$, and create the following matrix of partial derivatives:

\begin{equation*}\begin{bmatrix}\frac{\partial \Delta R}{\partial R_{s}}&\frac{\partial \Delta R}{\partial L_{s}} & \frac{\partial \Delta R}{\partial P_{c}} & \frac{\partial \Delta R}{\partial T_{c}}\\
\frac{\partial \Delta L}{\partial R_{s}}&\frac{\partial \Delta L}{\partial L_{s}}&\frac{\partial \Delta L}{\partial P_{c}} & \frac{\partial \Delta L}{\partial T_{c}}\\
\frac{\partial \Delta P}{\partial R_{s}}&\frac{\partial \Delta P}{\partial L_{s}} & \frac{\partial \Delta P}{\partial P_{c}} & \frac{\partial \Delta P}{\partial T_{c}}\\
\frac{\partial \Delta T}{\partial R_{s}}&\frac{\partial \Delta T}{\partial L_{s}}&\frac{\partial \Delta T}{\partial P_{c}}&\frac{\partial \Delta T}{\partial T_{c}}
\end{bmatrix}\end{equation*}

We will then solve the following equation, which is essentially (partial derivatives $\times$ adjustments $ = -$ mismatches), to find the adjustments vector.

\begin{equation*}
\begin{bmatrix}
\frac{\partial \Delta R}{\partial R_{s}}&\frac{\partial \Delta R}{\partial L_{s}} & \frac{\partial \Delta R}{\partial P_{c}} & \frac{\partial \Delta R}{\partial T_{c}}\\
\frac{\partial \Delta L}{\partial R_{s}}&\frac{\partial \Delta L}{\partial L_{s}} & \frac{\partial \Delta L}{\partial P_{c}} & \frac{\partial \Delta L}{\partial T_{c}}\\
\frac{\partial \Delta P}{\partial R_{s}}&\frac{\partial \Delta P}{\partial L_{s}} & \frac{\partial \Delta P}{\partial P_{c}} & \frac{\partial \Delta P}{\partial T_{c}}\\
\frac{\partial \Delta T}{\partial R_{s}}&\frac{\partial \Delta T}{\partial L_{s}} & \frac{\partial \Delta T}{\partial P_{c}}&\frac{\partial \Delta T}{\partial T_{c}}
\end{bmatrix}\begin{bmatrix}A(R_{s})\\A(L_{s})\\A(P_{c})\\A(T_{c})\end{bmatrix}\end{equation*}
\begin{equation*}
=-\begin{bmatrix}\Delta R \\ \Delta L \\ \Delta P \\ \Delta T\end{bmatrix}
\end{equation*}

We can do this by calculating the inverse of the partial derivatives matrix and multiplying it by the mismatch vector:

\begin{bmatrix}
\frac{\partial \Delta R}{\partial R_{s}}&\frac{\partial \Delta R}{\partial L_{s}} & \frac{\partial \Delta R}{\partial P_{c}} & \frac{\partial \Delta R}{\partial T_{c}}\\
\frac{\partial \Delta L}{\partial R_{s}}&\frac{\partial \Delta L}{\partial L_{s}}&\frac{\partial \Delta L}{\partial P_{c}} & \frac{\partial \Delta L}{\partial T_{c}}\\
\frac{\partial \Delta P}{\partial R_{s}}&\frac{\partial \Delta P}{\partial L_{s}} & \frac{\partial \Delta P}{\partial P_{c}} & \frac{\partial \Delta P}{\partial T_{c}}\\
\frac{\partial \Delta T}{\partial R_{s}}&\frac{\partial \Delta T}{\partial L_{s}}&\frac{\partial \Delta T}{\partial P_{c}}&\frac{\partial \Delta T}{\partial T_{c}}
\end{bmatrix}^{-1}

\begin{bmatrix}
\frac{\partial \Delta R}{\partial R_{s}}&\frac{\partial \Delta R}{\partial L_{s}} & \frac{\partial \Delta R}{\partial P_{c}} & \frac{\partial \Delta R}{\partial T_{c}}\\
\frac{\partial \Delta L}{\partial R_{s}}&\frac{\partial \Delta L}{\partial L_{s}}&\frac{\partial \Delta L}{\partial P_{c}} & \frac{\partial \Delta L}{\partial T_{c}}\\
\frac{\partial \Delta P}{\partial R_{s}}&\frac{\partial \Delta P}{\partial L_{s}} & \frac{\partial \Delta P}{\partial P_{c}} & \frac{\partial \Delta P}{\partial T_{c}}\\
\frac{\partial \Delta T}{\partial R_{s}}&\frac{\partial \Delta T}{\partial L_{s}}&\frac{\partial \Delta T}{\partial P_{c}}&\frac{\partial \Delta T}{\partial T_{c}}
\end{bmatrix}

\begin{bmatrix}A(R_{s})\\A(L_{s})\\A(P_{c})\\A(T_{c})\end{bmatrix}=\begin{bmatrix}A(R_{s})\\A(L_{s})\\A(P_{c})\\A(T_{c})\end{bmatrix}=-\begin{bmatrix}
\frac{\partial \Delta R}{\partial R_{s}}&\frac{\partial \Delta R}{\partial L_{s}} & \frac{\partial \Delta R}{\partial P_{c}} & \frac{\partial \Delta R}{\partial T_{c}}\\
\frac{\partial \Delta L}{\partial R_{s}}&\frac{\partial \Delta L}{\partial L_{s}}&\frac{\partial \Delta L}{\partial P_{c}} & \frac{\partial \Delta L}{\partial T_{c}}\\
\frac{\partial \Delta P}{\partial R_{s}}&\frac{\partial \Delta P}{\partial L_{s}} & \frac{\partial \Delta P}{\partial P_{c}} & \frac{\partial \Delta P}{\partial T_{c}}\\
\frac{\partial \Delta T}{\partial R_{s}}&\frac{\partial \Delta T}{\partial L_{s}}&\frac{\partial \Delta T}{\partial P_{c}}&\frac{\partial \Delta T}{\partial T_{c}}
\end{bmatrix}^{-1}\begin{bmatrix}\Delta R \\ \Delta L \\ \Delta P \\ \Delta T\end{bmatrix}

We will use `numpy.linalg.inv` to get the inverse of the partial derivatives matrix. We can then add the values from the adjustments vector to our initial values of $R_s$, $L_s$, $P_c$, and $T_c$, and run the whole program again. This should result in the mismatches getting smaller.

Note that overly large adjustments to the input values can crash the program. For this reason, our code will take the partial derivatives matrix and the mismatch vector as inputs, calculate the adjustments vector, and then add a fraction of the adjustments vector (say $25\%$) to our input values. This will happen inside a while loop.

In [ ]:
def improve(rlpt, do_plot):
    
    delta_rlpt, step_rlpt = model(rlpt, do_plot)
    
    fractional_change = 0.0001
    
    delta_rlpt_rs, step_rlpt_rs = model(rlpt * [1+fractional_change, 1, 1, 1], False)
    delta_rlpt_ls, step_rlpt_ls = model(rlpt * [1, 1+fractional_change, 1, 1], False)
    delta_rlpt_pi, step_rlpt_pi = model(rlpt * [1, 1, 1+fractional_change, 1], False)
    delta_rlpt_ti, step_rlpt_ti = model(rlpt * [1, 1, 1, 1+fractional_change], False)
    
    drlpt_drs = (delta_rlpt_rs - delta_rlpt) / (fractional_change * rlpt[0])
    drlpt_dls = (delta_rlpt_ls - delta_rlpt) / (fractional_change * rlpt[1])
    drlpt_dpi = (delta_rlpt_pi - delta_rlpt) / (fractional_change * rlpt[2])
    drlpt_dti = (delta_rlpt_ti - delta_rlpt) / (fractional_change * rlpt[3])
        
    mat = np.matrix([drlpt_drs, drlpt_dls, drlpt_dpi, drlpt_dti])
    
    adj = - np.linalg.inv(mat.T) * np.array([delta_rlpt]).T
    
    return np.abs(rlpt + 0.25 * adj.A1), delta_rlpt, step_rlpt

Now we are finally ready to try to model the star! There are a few ways we can assess how appropriate the model is and to decide when to stop iterating. The most thorough of these is to run our loop until the mismatches in each quantity ($\Delta R$, $\Delta L$, $\Delta P$, and $\Delta T$) at the meeting point are smaller than the change in each quantity between the meeting point and the extra integration step.

In [ ]:
# Print a nice heading for the text output

print("Iter   Radius          Luminosity      Core pressure   Core temperature")

# Start with an initial guess of the four boundary conditions

m0 = np.array([10**log_rs_i, 10**log_ls_i, 10**log_pc_i, 10**log_tc_i])
np.set_printoptions(formatter={'float_kind':"{: .8e}".format})
print("{0: 4d}".format(0), m0)

# Loop until mismatch values are smaller than the last-step changes in the same quantity

i=0
fraction = 10
while np.any(fraction > 1):
    m0, delta_rlpt, step_rlpt = improve(m0, False)
    fraction = np.abs(delta_rlpt/step_rlpt)
    
    i+=1
    print("{0:4d}".format(i), m0)

# Do one last iteration and plot the results
    
improve(m0, True)

This is great, but it can take quite a long time because there are lots of iterations and our code is far from optimised. We could do some work to improve the model code and make it faster, but this is beyond the scope of this activity. For now I'll simply present two alternative tolerances that may allow us to compute the model faster.

The first of these iterates until the fractional changes of all variables with respect to our initial values are below $0.5\%$.

In [1]:
# Print a nice heading for the text output

print("Iter   Radius          Luminosity      Core pressure   Core temperature")

# Start with an initial guess of the boundary conditions

m0 = np.array([10**log_rs_i, 10**log_ls_i, 10**log_pc_i, 10**log_tc_i])
np.set_printoptions(formatter={'float_kind':"{: .8e}".format})
print("{0: 4d}".format(0), m0)
init_m0 = np.copy(m0)

# Loop until fractional changes of all variables w.r.t. initial values are below 0.5% 

i = 0
fractional_change = np.array([100, 100, 100, 100])
while np.any(np.abs(fractional_change) > 0.5):
    m0, delta_rlpt, step_rlpt = improve(m0, False)
    fractional_change = np.array(delta_rlpt)*100/init_m0
    
    i+=1
    print("{0:4d}".format(i), m0)

# Do one last iteration and plot the results

improve(m0, True)

Iter   Radius          Luminosity      Core pressure   Core temperature


NameError: name 'np' is not defined

This was certainly faster - the number of iterations was reduced by a factor of almost 3 - but we can see there is still a slight mismatch in the luminosity integrations so the model isn't quite as good.

The final option is to loop for a fixed number of iterations. This gives us far greater control over how long the code takes to run, but the results may not be so great. This can also be a helpful approach if we want to see what is happening at a particular point in our process.

In [ ]:
# Print a nice heading for the text output

print("Iter   Radius          Luminosity      Core pressure   Core temperature")

# Start with an initial guess of the boundary conditions

m0 = np.array([10**log_rs_i, 10**log_ls_i, 10**log_pc_i, 10**log_tc_i])
np.set_printoptions(formatter={'float_kind':"{: .8e}".format})
print("{0: 4d}".format(0), m0)

# Loop for a set number of iterations (currently 20)

for i in range(1,21):
    m0, delta_rlpt, step_rlpt = improve(m0, False)
    
    print("{0:4d}".format(i), m0)

# Do one last iteration and plot the results

improve(m0, True)

This did ok but clearly the luminosity remains flawed, and there is a bit of a bump in temperature and density. Where possible we should stick with the first method.